In [611]:
from binance.client import Client, AsyncClient
import pandas as pd
import numpy as np
import math
import asyncio
import os
from coin_data import data
from config import api_key, api_secret, telegram_token
import requests
import nest_asyncio
import json
from datetime import datetime
from colorama import init, Fore, Back, Style

In [612]:
nest_asyncio.apply()
init()

In [613]:
datetime.now().strftime("%d - %b - %y , %I:%M:%S %p")
#datetime.now().strftime('%A %d-%m-%Y, %H:%M:%S')

'11 - Aug - 22 , 12:33:13 AM'

In [614]:
with open("./coins.json", "r", encoding='utf-8') as f:
    coins = json.load(f)
#print(coins)

In [615]:
def enviar_señal(msg: str):
    token = telegram_token
    bot_chatID = "1357832345"
    base = "https://api.telegram.org/bot"
    send_url = base + token + '/sendMessage'
    data = {
        'chat_id': bot_chatID,
        'text': msg
    }
    response = requests.post(send_url, json=data)
    return response

In [616]:
async def get_order_book(client: AsyncClient, symbol: str) -> dict:
    client = Client(api_key=api_key, api_secret=api_secret)
    order_book = client.futures_order_book(symbol=symbol, limit=1000)
    return order_book

In [617]:
async def get_data_from_order_book(order_book: dict, trx: str) -> pd.DataFrame:
    df = order_book[trx]
    if trx == "bids":
        df = pd.DataFrame(df, columns=['precio', 'total_usdt'])
    else:
        df = pd.DataFrame(df, columns=['precio', 'total_usdt']).sort_index(
            axis=0, ascending=False)
    df[['precio', 'total_usdt']] = df[[
        'precio', 'total_usdt']].astype('float')
    return df

In [618]:
def get_max_min(df: pd.DataFrame) -> list:
    max = df['precio'].max()
    min = df['precio'].min()
    return [min, max]

In [619]:
def crear_rango(df: pd.DataFrame, delta: float, tipo: str) -> np.arange:
    min, max = get_max_min(df)

    try:
        if tipo.lower() == 'c':
            inicio = (math.floor((min / delta))) * delta
            final = (math.ceil((max / delta))) * delta
            rango = np.arange(inicio, final+delta, delta)
        elif tipo.lower() == 'v':
            inicio = (math.floor((min / delta))) * delta
            final = (math.ceil((max / delta))) * delta
            rango = np.arange(inicio, final+delta, delta)
        else:
            raise ValueError(
                'Ingresa (c) si es rango de compra o (v) si es rango de venta')
    except ValueError as ve:
        print(ve)

    return rango

In [620]:
def get_shock_point(df: pd.DataFrame, rango: np.arange, tipo: str, vuelta: int) -> float:
    df_ag = df.groupby(pd.cut(df.precio, rango)).sum()
    if tipo == 'v' and vuelta == 1:
        sp = df_ag['total_usdt'].idxmax().right
    if tipo == 'v' and vuelta == 2:
        sp = df_ag['total_usdt'].idxmax().right
    elif tipo == 'c' and vuelta == 1:
        sp = df_ag['total_usdt'].idxmax().left
    elif tipo == 'c' and vuelta == 2:
        sp = df_ag['total_usdt'].idxmax().left
   
    return sp

In [621]:
async def get_shock_points(ticker: str, ventas: pd.DataFrame, compras: pd.DataFrame) -> dict:
    sp_v_1 = get_shock_point(ventas, crear_rango(
        ventas, data[ticker]['i1'], 'v'), 'v', 1)
    sp_v_2 = get_shock_point(ventas, crear_rango(
        ventas, data[ticker]['i2'], 'v'), 'v', 2)
    sp_c_1 = get_shock_point(compras, crear_rango(
        compras, data[ticker]['i1'], 'c'), 'c', 1)
    sp_c_2 = get_shock_point(compras, crear_rango(
        compras, data[ticker]['i2'], 'c'), 'c', 2)
    spv = sorted([sp_v_1, sp_v_2])
    spc = sorted([sp_c_1, sp_c_2])
    
    
    return {
        'ventas': spv,
        'compras': spc
    }

In [622]:
async def analizar_shock_points(shock_points: dict, ticker:str, sl=0.2) -> bool:
    flag_venta = False
    flag_compra = False

    sp_v_m = shock_points['ventas'][0]
    sp_v_M = shock_points['ventas'][1]
    sp_c_m = shock_points['compras'][0]
    sp_c_M = shock_points['compras'][1]

    pct_ventas = round(((sp_v_M - sp_v_m) / sp_v_m) * 100, 2)
    pct_compras = round(((sp_c_M - sp_c_m) / sp_c_M) * 100, 2)
    pct_sep_1 = (((sp_c_M - sp_v_m) / sp_c_M) * -100)
    pct_sep_2 = (((sp_v_m - sp_c_M) / sp_v_m) * 100)
    pct_sep = round((pct_sep_1+pct_sep_2)/2, 2)
    
    print('pct_sep')
    print(type(pct_sep))
    print(pct_sep)
    print(f'{"+"*10}')
    print('pct_ventas')
    print(type(pct_ventas))
    print(pct_ventas)
    print(f'{"+"*10}')
    print('sl')
    sl = np.float64(sl)
    print(type(sl))
    print(sl)
    print(f'{"+"*10}')
    div_1 = pct_sep/(pct_ventas+sl) 
    print('div1')
    print(div_1)
    
    if (div_1 >= 2) and (sp_v_m != sp_v_M):  # Aquí es pct_sep/(pct_ventas+SL)
        #print(f'Se puede vender en {ticker}')
        flag_venta = True
    if ((pct_sep/(pct_compras+sl)) >= 2) and (sp_c_m != sp_c_M):  # Aquí es pct_sep/(pct_compras+SL)
        #print(f'Se puede comprar en {ticker}')
        flag_compra = True

    # return flag_venta, flag_compra

    signal_venta = {
        'signal': flag_venta,
        'sp_v_M': sp_v_M,
        'sp_v_m': sp_v_m
    }
    signal_compra = {
        'signal': flag_compra,
        'sp_c_M': sp_c_M,
        'sp_c_m': sp_c_m
    }
    return signal_venta, signal_compra


In [623]:
async def run(ticker, client):
    flagEj = True
    while flagEj:
        order_book = await get_order_book(client, symbol=ticker)
        ventas, compras = await asyncio.gather(
            get_data_from_order_book(order_book, 'asks'),
            get_data_from_order_book(order_book, 'bids')
        )
        shock_points = await get_shock_points(ticker, ventas, compras)
        #signal_venta, signal_compra = await analizar_shock_points(shock_points)
        #if signal_venta and signal_compra:
        #    #print(f'señal de venta y compra en {ticker}\n{shock_points}')
        #    enviar_señal(
        #        f'señal de venta y compra en {ticker}\n{shock_points}')
        #    break
        #if signal_venta and not signal_compra:
        #    #print(f'señal de venta en {ticker}\n{shock_points["ventas"]}')
        #    enviar_señal(
        #        f'señal de venta en {ticker}\n{shock_points["ventas"]}')
        #    break
        #if not signal_venta and signal_compra:
        #    #print(f"señal de compra en {ticker}\n {shock_points['compras']}")
        #    enviar_señal(
        #        f"señal de compra en {ticker}\n {shock_points['compras']}")
        #    break
        
        signal_venta, signal_compra = await analizar_shock_points(shock_points, ticker)
        if signal_venta['signal'] and signal_compra['signal']:
            #print(f'señal de venta y compra en {ticker}\n{shock_points}')
            texto = f"{datetime.now().strftime('%d - %b - %y , %I:%M:%S %p')}\n{Fore.YELLOW}Señal de Compra y Veta\n{ticker}\n{'*'*10}\n{Style.RESET_ALL}\n{Fore.RED}Puntos Venta:{Style.RESET_ALL}\n2- {signal_venta['sp_v_M']}\n1- {signal_venta['sp_v_m']}\n\n{Fore.GREEN}Puntos Compra:\n1- {signal_compra['sp_c_M']}\n2- {signal_compra['sp_c_m']}{Style.RESET_ALL}"
            #enviar_señal(texto)
            print(texto)
            
        if signal_venta['signal'] and not signal_compra['signal']:
            texto_v = f"{Fore.RED}Señal de Venta\n{'*'*10}\n{ticker}\n{Style.RESET_ALL}\n{Fore.RED}Puntos Venta:\n2- {signal_venta['sp_v_M']}\n1- {signal_venta['sp_v_m']}{Style.RESET_ALL}\n\nPuntos Compra:\n1- {signal_compra['sp_c_M']}\n2- {signal_compra['sp_c_m']}"
            #enviar_señal(f"{datetime.now().strftime('%d - %b - %y , %I:%M:%S %p')}\n{signal_venta['msg']}")
            print(texto_v)
            
        if not signal_venta and signal_compra:
            #enviar_señal(f"{datetime.now().strftime('%d - %b - %y , %I:%M:%S %p')}\nsignal_compra['msg']")
            texto_c = f"{Fore.GREEN}Señal de Compra\n{'*'*10}\n{ticker}\n{Style.RESET_ALL}\nPuntos Venta:\n2- {signal_venta['sp_v_M']}\n1- {signal_venta['sp_v_m']}\n\n{Fore.GREEN}Puntos Compra:\n1- {signal_compra['sp_c_M']}\n2- {signal_compra['sp_c_m']}{Style.RESET_ALL}"
            print(texto_c)
        flagEj = False

In [624]:
async def main():
    client = await AsyncClient.create()
    #tickers = coins
    tickers = ['BTCUSDT']
    tasks = []
    for t in tickers:
        tasks.append(asyncio.create_task(run(t, client)))
    await asyncio.gather(*tasks)
    await client.close_connection()

In [625]:
#if __name__ == '__main__':
#    asyncio.run(main())
asyncio.run(main())

pct_sep
<class 'numpy.float64'>
0.61
++++++++++
pct_ventas
<class 'numpy.float64'>
0.0
++++++++++
sl
<class 'numpy.float64'>
0.2
++++++++++
div1
3.05
